<a href="https://colab.research.google.com/github/MahenderTanniru/chatbot_ML/blob/main/RESTURANTBOT(BABAI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flask-ngrok

In [ ]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [ ]:
with open('intents.json') as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [ ]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [ ]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

In [ ]:
epochs = 500
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/500
2/2 [==============================] - 1s 19ms/step - loss: 2.1978 - accuracy: 0.1212
Epoch 2/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1967 - accuracy: 0.1212
Epoch 3/500
2/2 [==============================] - 0s 5ms/step - loss: 2.1963 - accuracy: 0.1212
Epoch 4/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1956 - accuracy: 0.1212
Epoch 5/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1950 - accuracy: 0.1212
Epoch 6/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1945 - accuracy: 0.1212
Epoch 7/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1939 - accuracy: 0.1212
Epoch 8/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1932 - accuracy: 0.1212
Epoch 9/500
2/2 [==============================] - 0s 7ms/step - loss: 2.1927 - accuracy: 0.1212
Epoch 10/500
2/2 [==============================] - 0s 6ms/step - loss: 2.1920 - accuracy: 0.1212
Epoch 11/500
2/2 [==========

In [ ]:
model.save("chat_model")

import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
with open('label_encoder.pickle', 'wb') as ecn_file:
  pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model/assets


In [ ]:
pip install colorama

In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
import colorama 
colorama.init()
from colorama import Fore, Style, Back
import random
import pickle

with open("intents.json") as file:
  data = json.load(file)
model = keras.models.load_model('chat_model')
with open('tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)
with open('label_encoder.pickle', 'rb') as enc:
  lbl_encoder = pickle.load(enc)
max_len = 20


app = Flask(__name__,template_folder='/content')
run_with_ngrok(app) 

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/get")

def get_bot_response():
    userText = request.args.get('msg')
    result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([userText]),truncating='post', maxlen=max_len))
    tag = lbl_encoder.inverse_transform([np.argmax(result)])
    for i in data['intents']:
      if i['tag'] == tag:
        return str(np.random.choice(i['responses']))

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://2f9d-35-204-193-43.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [13/May/2022 15:55:36] "GET / HTTP/1.1" 200 -
